In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

from constants import exclude_models, model_config_file
from helper import load_model_configs_and_allowed_models, load_similarity_matrices

sys.path.append('..')
from scripts.helper import parse_datasets
from constants import sim_metric_name_mapping

In [ ]:
base_path_similarity_matrices = Path('/home/space/diverse_priors/model_similarities')

sim_metrics = [
    'cka_kernel_rbf_unbiased_sigma_0.4',
    'cka_kernel_linear_unbiased',
]
sim_metrics_mapped = [sim_metric_name_mapping[k] for k in sim_metrics]

ds_list = parse_datasets('../scripts/webdatasets_w_insub10k.txt')
ds_list = list(map(lambda x: x.replace('/', '_'), ds_list))

storing_path = Path('/home/space/diverse_priors/results/aggregated/model_sims')
storing_path.mkdir(parents=True, exist_ok=True)

In [ ]:
model_configs, allowed_models = load_model_configs_and_allowed_models(
    path=model_config_file,
    exclude_models=exclude_models,
    exclude_alignment=True,
)
print(model_configs.shape)

In [ ]:
sim_mats = load_similarity_matrices(
    path=base_path_similarity_matrices,
    ds_list=ds_list,
    sim_metrics=sim_metrics,
    allowed_models=allowed_models,
)
sim_mats = {sim_metric_name_mapping[k]: v for k, v in sim_mats.items()}

In [ ]:
def flatten_sim_values(sim_mat):
    iu2 = np.triu_indices(sim_mat.shape[0], k=1)
    flat_sim_mat = pd.DataFrame({
        'Model 1': sim_mat.index.values[iu2[0]],
        'Model 2': sim_mat.columns.values[iu2[1]],
        'Similarity value': sim_mat.values[iu2],
    })
    return flat_sim_mat


def sort_tuple(tup):
    return tuple(sorted(tup))


def get_cat_pair(row, cat):
    pair_info = (
        model_configs.loc[row['Model 1'], cat],
        model_configs.loc[row['Model 2'], cat]
    )
    return sort_tuple(pair_info)


def process_sim_mat(sim_mat, metric, ds):
    flat_sim_mat = flatten_sim_values(sim_mat)
    flat_sim_mat['Similarity metric'] = metric
    flat_sim_mat['DS'] = ds
    flat_sim_mat['Objective pair'] = flat_sim_mat.apply(get_cat_pair, axis=1, cat='objective')
    flat_sim_mat['Architecture pair'] = flat_sim_mat.apply(get_cat_pair, axis=1, cat='architecture_class')
    flat_sim_mat['Dataset pair'] = flat_sim_mat.apply(get_cat_pair, axis=1, cat='dataset_class')
    flat_sim_mat['Model size pair'] = flat_sim_mat.apply(get_cat_pair, axis=1, cat='size_class')
    cols = flat_sim_mat.columns.tolist()
    flat_sim_mat = flat_sim_mat[cols[3:5] + cols[:3] + cols[5:]]
    return flat_sim_mat


def get_similarity_dataframe(similarity_matrices):
    dfs = []
    for sim_metric, sim_mats_w_metric in similarity_matrices.items():
        for ds, curr_sim_mat in sim_mats_w_metric.items():
            dfs.append(process_sim_mat(curr_sim_mat, sim_metric, ds))
    df = pd.concat(dfs)
    return df

In [ ]:
sim_df = get_similarity_dataframe(sim_mats)

In [ ]:
sim_df.to_csv(storing_path / 'all_metric_ds_model_pair_similarity.csv', index=False)